# Understanding Color Blindness Simulations and their Open Source implementations
> Let's stop copy/pasting code without understanding where it comes from :-)

- toc: true 
- badges: true
- comments: true
- categories: [jupyter]
- image: images/chart-preview.png

In [ ]:
# hide
%load_ext autoreload
%autoreload 2

In [ ]:
# hide
%pip install -q plotly pandas Geometry3D colour-science
# %pip install -q daltonlens

In [ ]:
# hide_input
import plotly.io as pio
pio.renderers.default = 'notebook_connected'
from IPython.display import display, HTML
js = '<script src="https://cdnjs.cloudflare.com/ajax/libs/require.js/2.3.6/require.min.js" integrity="sha512-c3Nl8+7g4LMSTdrm621y7kf9v3SDPnhxLNhcjFJbKECVnmZHTdo+IRO05sNLTH/D3vA6u1X32ehoLC7WFVdheg==" crossorigin="anonymous"></script>'
display(HTML(js))

# Introduction

## Why simulate color vision deficiencies (CVD)?

Color vision deficiencies (CVD), commonly called "color blindness" affects ~8% of the male population and ~0.4% of the female population. Being able to accurately simulate them is important for several reasons:
- Communicate about this issue and help other people understand it
- Help designers choose color schemes that everyone will perceive
- Create tools to enhance images and help color blind people in their daily tasks. Most correction tools start by simulating how a person with CVD would see the image, and then find some way to spread the lost information into the channels that are better perceived or playing with the light intensity to restore the contrast.

There are many online resources to understand CVD in detail. [color-blindness.com](https://color-blindness.com/) is a good start, but here are the most important facts:

- Human color perception is achieved via cone cells in the retina. Humans with normal vision have 3 kinds of cells, sensitive to different light wavelengths. L cones capture short-wavelength (red), M cones capture medium-wavelength (green) and S cones capture long-wavelength (blue). Their response over the light spectrum is shown below.

![image](https://upload.wikimedia.org/wikipedia/commons/d/d1/Cone_spectral_sensitivities.png "LMS Response over the light wavelength (from Wikipedia). L is roughly centered on red, M on green and S on blue. Note that the responses overlap significantly, especially for L and M cones.").

- Most color vision deficiencies can be explained by one type of cone not working properly. Protanopes, deuteranopes and tritanopes respectively lack or have a malfunctioning L, M, or S cone cell.

- CVD simulations based on physiological experiments generally consists in transforming the image to a color space where the influence of each cone cell is explicit and can be decreased or removed easily. The [LMS](https://en.wikipedia.org/wiki/LMS_color_space) color space was designed to specifically match the human cone responses and is thus the choice of the vast majority of methods.

## Making sense of the available models and programs

There are many methods to simulate color blindness, and the web has lots of opensource programs to do so. However color appearance theory is complex and most software engineers (like me!) end up copying existing algorithms without having a solid understanding of where they come from and which one is best.

While developing [DaltonLens](daltonlens.org) I got frustrated as I was trying to decide which method I should implement. And being a mild-protan myself, I was often not very convinced by the results of existing methods as they tended to make the simulated images way too exaggerated.

So I decided to dig further into the history of the available algorithms and try to understand where they come from and how they work. To visualize and play with the models interactively this page is actually originally a [Jupyter notebook](https://github.com/DaltonLens/DaltonLens-Python/blob/master/research/understanding-simulations/UnderstandingColorBlindness.ipynb) that can be run in e.g. Google colab to play with it.

## Vocabulary notes

As it can quickly get confusing, here is a summary of the main terms to describe CVD variants:

- *Dichromacy* refers to one cone fully missing. The corresponding deficiencies for each cone are called *protanopia*, *deuteranopia* and *tritanopia*. The color vision of people with dichromacy (*dichromats*) is basically 2D instead of 3D and simulations of full dichromacy generally end up projecting the 3D color space to a carefully chosen 2D plane. This contrasts with *trichromacy* which is the normal vision for humans. As a side note some animals (e.g. goldfishes) have 4 types of cones and are [tetrachromats](https://en.wikipedia.org/wiki/Tetrachromacy). 

- *Anomalous trichromacy* refers to having one cone with only a partial disfunction (e.g less density or with a shifted wavelenght peak). The corresponding deficiencies are called *protanomaly*, *deuteranomaly* and *tritanomaly*. In simulations the degree of severity is usually encoded as a float value between 0 (no deficiency) and 1 (dichromacy).

- A *protan* or a *protanope* is a person suffering from protanopia ("strong" protan) or protanomaly ("mild" protan). *Deutan/deuteranope* and *tritan/tritanope* are defined similarly.

# Overview of the existing approaches

Let's start by looking at the most popular software and research papers. To complete this section I used the nice thread and software links compiled by Markku Laine in a [pull-request of the Aalto Interface Metrics (AIM) project](https://github.com/aalto-ui/aim/pull/13).

## Coblis and the HCIRN Color Blind Simulation function

A google search for "color blindness simulation" first returns [color-blindness.com](https://color-blindness.com/). I mentioned that website before because it has great introductory material, but it also proposes a CVD simulator, called *Coblis*. Being one of the oldest and easiest tool to test it has inspired lots of other software. But digging into the history of the code is interesting and shows that its accuracy is quite questionable.

The simulator relies on the source code of MaPePeR https://github.com/MaPePeR/jsColorblindSimulator . It implements two different functions, one based on the "Color-Matrix" algorithm, and one based on the "HCIRN Color Blind Simulation function".

The "Color-Matrix" algorithm was developed by [www.colorjack.com](http://web.archive.org/web/20081014161121/http://www.colorjack.com/labs/colormatrix/). He converted the "HCIRN Color Blind Simulation function" that worked in the CIE-XYZ color-space into a faster matrix that directly works on RGB values (that kind of simplification mattered back then). He did so by running the full function on 3 RGB values (pure red, pure green, pure blue) to directly read the RGB matrices. So this approach only ensures the equivalence for these 3 colors. It also ignores the non-linearity of sRGB images, which was approximately handled in the original code a gamma function (more about sRGB in section FIXME). The author himself later said that this was a one-night hack and that nobody should use it anymore in a [comment on kaioa.com](https://web.archive.org/web/20111123145815/http://kaioa.com/node/75#comment-247):
> You're right, the ColorMatrix version is very simplified, and not accurate. I created that color matrix one night (http://www.colorjack.com/labs/colormatrix/) and since then it's shown up many places... I should probably take that page down before it spreads more! Anyways, it gives you an idea of what it might look like, but for the real thing...

The author actually took down that page since then, but it's hard to stop the spread.

Regarding the proper "HCIRN Color Blind Simulation function", the first public code was developed by [Matthew Wickline](http://colorlab.wickline.org/colorblind/colorlab/docs/acknowledgments.html) and made available in a javascript [Color Blindness Library](https://web.archive.org/web/20071012020140/http://www.nofunc.com/Color_Blindness_Library/). The author says in the acknowledgments that he wrote the code by adapting some Java code that he kindly got from Thomas Wolfmaier after reading his 1999 article [Designing for the Color-Challenged: A Challenge](https://web.archive.org/web/20071212234539/http://www.internettg.org/newsletter/mar99/accessibility_color_challenged.html). 

This article explains that they implemented a method based on the seminal work of {% cite meyer_color_defective_1988 %}, which was the first to propose an algorithm to simulate CVD, in the CIE XYZ color space. However the article of Thomas Wolfmaier was not peer-reviewed and their implementation was not validated experimentally, here are their conclusions in the article:
> Has the model been validated on individuals with color vision deficiencies? Unfortunately, we have not yet been able to test the model. We did apply the model to some of the tables of the UMIST 'For Fun' Colour Vision Test and were able to reproduce the predicted confusions. If you have some form of color vision deficiency, please have a look at the design aids described in the following sections and let us know how well the model predicts the color you see. Please send your feedback to Thomas Wolfmaier.

> How accurate is the model? The model provides only a rough approximation. It includes estimates for some properties of color vision defects as well as assumptions about the hardware on which the colors are displayed. It also does not account for the reduced sensitivity to reds of individuals with protan defects.

It is worth noting that {% cite meyer_color_defective_1988 %} did some experimental validation, where they noted that dichromats responded favorably to the simulation besides some issues with highly saturated colors. {% cite vienot_digital_1999 %} later stated that working in the CIE XYZ color space is worse as it does not take into account the altered perception of luminosity for dichromats.

So to summarize:

- The ColorMatrix algorithm that was used in the first version of Coblis was a one-night hack severely simplifying the Javascript code of Matthew Wickline, which itself was an adaptation of the Java code of Thomas Wolfmaier in 1999. The author of the ColorMatrix himself said that we should not use it and removed the code, so let's stop using that.

- The code adapted by Matthew Wickline from Thomas Wolfmaier article was inspired by the Meyer and Greeberg paper which was a solid work. But the code itself was not validated and the experimental validation in the 1988 paper was not fully convincing and somewhat outdated. So it's unclear whether we can trust it, and the more recent academic litterature by colour science researchers moved to more recent approaches. In any case that code should be adapted to modern monitors by implementing the proper sRGB transform from RGB to CIE XYZ as the original code used a generic gamma correction (there was no sRGB standard back then). The code with the generic gamma is the version currently used by default in Coblis v2 (as of 2021), based on MaPePeR's implementation.

- Last, the license of the code is not super permissive, it cannot be used for commercial applications.

Other examples of software using the HCIRN Color Blind Simulation function or the ColorMatrix are: 
- [Peacock (Python & C++)](https://github.com/jkulesza/peacock). HCIRN Color Blind Simulation function.
- [Colorblindly (chrome extension)](https://github.com/oftheheadland/Colorblindly). Uses the ColorMatrix.

## Brettel & Mollon 1997

A more modern algorithm for dichromacy simulation was developed by {% cite brettel_computerized_1997 %}. Their paper has become a reference in the research community. It is based on the LMS color space, which is designed to model human perception at the cone level. It was performed on an old calibrated CRT monitor, so implementations of this approach need to be adapted for modern LCD monitors and the more recent sRGB standard. The most famous implementation is the one of [www.vischeck.com](www.vischeck.com) in Java. It also had an online version, but the website is now broken. vischeck was used to generate the examples in the reference color science book {% cite fairchild2013color %}.

There are comparatively few opensource implementation of this algorithm as the follow-up {% cite vienot_digital_1999 %} paper from the same group proposed a simpler algorithm for protanopia and deuteranopia that is also more adapted to digital monitors. However that newer approach does not work well for tritanopia, so the Brettel & Mollon approach is still relevant in that case.

Another popular recent work from {% cite machado_physiologically_based_2009 %} uses Brettel & Molon as their reference and are actually very similar to it for dichromacy (they tuned their parameters to match it).

Note that even if it was initially developed for full dichromacy, the approach can be adapted to simulate less severe anomalous trichromacy by either linearly interpolating between the original image and the dichromat image or by applying smaller corrections with fixed steps ({% cite flatla_so_2012 %}). While reasonable in practice, the interpolation approach was not formally evaluated.

## Viénot, Brettel & Mollon 1999

{% cite vienot_digital_1999 %} builds on the Brettel & Mollon paper. We'll see more details in section FIXME:, but in a nutshell it simplifies the math and adapts it for digital displays. It has become a very popular reference, in large part thanks to the daltonize algorithm of {% cite fidaner_analysis_2005 %}. That daltonize algorithm aims at improving the color contrasts of a given image for a person with CVD. The first step is to simulate how the image is seen by a dichromat, and then distribute the error w.r.t the original image on the other color channels. That algorithm became very popular and got copy/pasted/adapted many times. Since it used the simulation algorithm of {% cite vienot_digital_1999 %} in the first step, this approach became popular along the way too.

For protanopia and deuteranopia the results are reasonably similar between Viénot 1999 and Brettel 1997, so the faster one can be preferred, but it is important to note that the Viénot paper is not well adapted to tritanopia. Indeed its simplifications do not hold very well in that case. The authors themselves only mention protanopia and  deuteranopia in the paper.

It can also be adapted for anomalous trichromacy like we discussed for the Brettel & Mollon approach, with the same limits about the validation.

One challenge when implementing that algorithm is that the original paper was also dealing with CRT monitors, and thus did not use the sRGB standard. So the RGB to XYZ matrices in the paper have to be adjusted for modern monitors, but many opensource code copied the original matrices from {% cite fidaner_analysis_2005 %} and are thus inaccurate. The XYZ to LMS transform is also subject to discussion and we'll talk about it in section FIXME.

A worse but also common issue is that the code from {% cite fidaner_analysis_2005 %} actually did not include any gamma decoding of the input image, so many software just apply the matrices on the raw RGB values of the image and skip the sRGB decoding step altogether.

Here are a few examples of opensource projects implementing it:

- [daltonize.py (Python)](https://github.com/joergdietrich/daltonize). Adapted to sRGB, and uses the CIECAM02 sharpened matrix for the RGB->LMS conversion. I think this is debatable for CVD simulation (see section FIXME).

- [Ixora.io (Processing)](https://ixora.io/projects/colorblindness/color-blindness-simulation-research/). Adapted to sRGB, uses the Hunt-Pointer-Estevez matrix for RGB->LMS, debatable for CVD simulation.

- [daltonize.org (several languages)](http://daltonize.org). Most software listed there are adaptation of {% cite fidaner_analysis_2005 %}. And most forget the sRGB decoding and still use the original CRT RGB->XYX conversion.

- [tsarjak/Simulate-Correct-ColorBlindness (Python)](https://github.com/tsarjak/Simulate-Correct-ColorBlindness). Also extends the approach to anomalous trichromacy by linearly interpolating between the original color and the dichromat color. Uses the original RGB<>LMS conversion for CRT monitors.

## Machado 2009

Even more recently {% cite machado_physiologically_based_2009 %} proposed an approach that supports both dichromacy and anomalous trichromacy in a principled way. They also made the nice and very welcome effort to publish easy-to-use matrices on [their website](https://www.inf.ufrgs.br/~oliveira/pubs_files/CVD_Simulation/CVD_Simulation.html) and it is becoming more and more popular.

As mentioned ealier it is actually very similar to Brettel & Mollon for dichromacy as they tuned their scaling parameters to match it. However it does not work very well for tritanopia as their model is to shift the peak response of the faulty cone, and how it should be shifted is unclear for tritanopes. They also did not do any experimental validation with tritanopes.

For anomalous trichromacy the results are reasonably similar to the variant of Brettel 1997 that interpolates with the original image, but since the Machado approach is more principled it can be preferred.

Here are some examples of software using this approach:

- [colour-science.org (Python)](https://github.com/colour-science/colour). Also has code to re-compute the predefined matrices. Overall a very comprehensive reference for anything color-related.

- [Chromium](https://bugs.chromium.org/p/chromium/issues/detail?id=1003700#c33) and [Mozilla](https://bugzilla.mozilla.org/show_bug.cgi?id=1655053) used to rely on the ColorMatrix (Coblis v1) algorithm but fortunately now rely on the Machado approach.

## So which one should we use?

Given the limits of the existing algorithms I would recommend different methods depending on the kind of deficiency:

- For tritanopia the Brettel 1997 approach is still the most solid and basically only valid choice. For tritanomaly I'd also recommend it with an interpolation factor with the original image, but this is more debatable. Calibrated fixed steps like {% cite flatla_so_2012 %} and {% cite macalpine_real_time_2016 %} are also an option.

- For protanopia and deuteranopia Viénot 1999, Brettel 1997 and Machado 2009 are solid choices, with a slight advantage for Viénot because it behaves a bit better with extreme values. For protanomaly and deuteranomaly Machado is more principled than linearly interpolating with the original image so it's probably a better choice.

This is the heuristic implemented by `Simulator_AutoSelect` in [DaltonLens-Python](https://github.com/DaltonLens/DaltonLens-Python).

More discussion and comparisons can be found in the {% cite simon_liedtke_using_2016 %} paper. It includes an evaluation of the {% cite kotera_optimal_2012 %} method, which I did not include as I haven't found any software implementing it (and it performs worse than Viénot and Brettel in that study).

## Show me some images!

We'll see the differences between some of these approaches in greater details later on, but let's get an initial idea of their output. The input image covers the full RGB range. These results are for full dichromacy, so people with a mild CVD will likely see significant differences between the original and the simulated images for all models.

The images were generated using the implemention of each method in [DaltonLens-Python](https://github.com/DaltonLens/DaltonLens-Python), with the exception of Coblis V1 and V2 which respectively come from [skratchdot/color-matrix](https://github.com/skratchdot/color-matrix) and [Peacock](https://github.com/jkulesza/peacock).

Overall it confirms that Coblis V1 (the ColorMatrix) is very broken. The other algorithms have some differences but fortunately still generally agree for protanopia and deuteranopia. For tritanopia Brettel 1997 is significantly different and should be more accurate as the other models were not designed to be compatible with tritanopia.

### For protanopia

<!-- Image tags have to be unindented for nbdev to catch them :-( -->
<table>
    <tr><th>Original</th><th>Machado 2009</th><th>Viénot 1999</th></tr>
    <tr><td>
<img src="simulation_images/rgbspan.png"></td><td>
<img src="simulation_images/rgbspan_protan_machado2009.png"></td><td>
<img src="simulation_images/rgbspan_protan_vienot1999.png"></td></tr>
     <tr><th>Brettel 1997</th><th>Coblis V1</th><th>Coblis V2</th></tr>
    <tr><td>
<img src="simulation_images/rgbspan_protan_brettel1997.png"></td><td>
<img src="simulation_images/rgbspan_protan_coblisv1.png"></td><td>
<img src="simulation_images/rgbspan_protan_coblisv2.png"></td></tr>
</table>

### For deuteranopia

<table>
    <tr><th>Original</th><th>Machado 2009</th><th>Viénot 1999</th></tr>
    <tr><td>
<img src="simulation_images/rgbspan.png"></td><td>
<img src="simulation_images/rgbspan_deutan_machado2009.png"></td><td>
<img src="simulation_images/rgbspan_deutan_vienot1999.png"></td></tr>
     <tr><th>Brettel 1997</th><th>Coblis V1</th><th>Coblis V2</th></tr>
    <tr><td>
<img src="simulation_images/rgbspan_deutan_brettel1997.png"></td><td>
<img src="simulation_images/rgbspan_deutan_coblisv1.png"></td><td>
<img src="simulation_images/rgbspan_deutan_coblisv2.png"></td></tr>
</table>

### For tritanopia

<table>
    <tr><th>Original</th><th>Machado 2009</th><th>Viénot 1999</th></tr>
    <tr><td>
<img src="simulation_images/rgbspan.png"></td><td>
<img src="simulation_images/rgbspan_tritan_machado2009.png"></td><td>
<img src="simulation_images/rgbspan_tritan_vienot1999.png"></td></tr>
     <tr><th>Brettel 1997</th><th>Coblis V1</th><th>Coblis V2</th></tr>
    <tr><td>
<img src="simulation_images/rgbspan_tritan_brettel1997.png"></td><td>
<img src="simulation_images/rgbspan_tritan_coblisv1.png"></td><td>
<img src="simulation_images/rgbspan_tritan_coblisv2.png"></td></tr>
</table>

# Understanding LMS-based approaches

{% cite brettel_computerized_1997 %}, {% cite vienot_digital_1999 %} and {% cite machado_physiologically_based_2009 %} all rely on the same low-level principles to simulate CVD. So we're going to focus on that pipeline and dig into it to understand how they should be implemented and get a sense of what they do. The Machado method is actually a bit more involved as it uses a second stage of modeling (opponent-color theory), so I will not cover it here and only focus on Viénot and Brettel.

These methods rely on transforming RGB images to the LMS color-space. The 3 axis of that space are meant to correspond to L cones (short-wavelength, red), M cones (medium-wavelength, green) and S cones (long-wavelength, blue).

Both Viénot 1999 and Brettel 1997 focus on full dichromacy, so we'll start with that. The extension to anomalous trichromacy will be discussed in section FIXME. Protanopes, deuteranopes and tritanopes respectively lack the L, M, and S cone cells. Thus a natural approach to simulate dichromacy is to go to the LMS color-space and somehow remove the component coming from the missing cone.

The good news is that going from an RGB color space to LMS can be done with a linear transformation (matrix multiplication). The bad news is that there are many ways to do it, and that will be the focus on section FIXME.

One extra complexity is that images are typically encoded in `sRGB` to get properly displayed on digital monitors once they apply their gamma correction. So our first step will be to go from sRGB to linear RGB, and then from there to LMS. Many open source programs forget that part.

Once in the LMS color space, we know that colors that only differ by the coordinate of the axis corresponding to the missing cone cannot be differentiated by a dichromat. This creates so-called confusion lines, which are, by construction, parallel to the missing axis in the LMS color space. So according to this model a protanope cannot distinguish colors that only differ by their L component, a deuteranope those that only differ by their M component, and tritanopes those that only differ by their S component.

To illustrate this let's build a plot with a few confusion lines in the LMS space, and their counterpart on the linear RGB space. We'll see the details on how to generate the confusion lines later on, but for now we'll just draw 5 precomputed lines, starting with protanopia (missing L cones). To have a reference we'll also add the gamut (range) of colors that can be produced by a digital monitor. It's a cube in the RGB color space, and a parallelepiped in the LMS space. The 8 vertices are black (`K = RGB[0,0,0]`), white (`W = RGB[1, 1, 1])`, red (`R = RGB[1, 0, 0])`, green (`G = RGB[0,1,0]`), blue (`B = RGB[0,0,1]`), yellow (`G = RGB[1,1,0]`), magenta (`M = RGB[1,0,1]`) and cyan (`C = RGB[0,1,1]`). 

> Note: the plots are in 3D and interactive, so feel free to play with the point of view!

In [ ]:
# collapse-hide
# This cell defines all our imports and a few utilities
import itertools
import numpy as np
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import pandas as pd
import Geometry3D as geo3d

import daltonlens.convert as convert
import daltonlens.generate as generate
import daltonlens.simulate as simulate
import daltonlens.geometry as geometry
np.set_printoptions(suppress=True)

def printMatrix(name, m):
    mat_str = '\n  '.join(np.array_repr(m).split('\n'))
    print (f'{name} = \n  {mat_str}')
    
# Utility for plotly imshow
hide_image_axes = dict(yaxis_visible=False, yaxis_showticklabels=False, xaxis_visible=False, xaxis_showticklabels=False)

In [ ]:
# collapse-hide

def show_confusion_lines(title, lines):
    rgb_cube = pd.DataFrame.from_records([
        ('K', 'black',     0,   0,   0, 0.        , 0.        , 0.        , '#000000'),
        ('B', 'blue',      0,   0,   1, 0.03596577, 0.03620635, 0.01528089, '#0000ff'),
        ('M', 'magenta',   1,   0,   1, 0.21482533, 0.07001029, 0.01559176, '#ff00ff'),
        ('R', 'red',       1,   0,   0, 0.17885956, 0.03380394, 0.00031087, '#ff0000'),
        ('G', 'green',     0,   1,   0, 0.43997117, 0.27515242, 0.00191661, '#00ff00'),
        ('C', 'cyan',      0,   1,   1, 0.47593694, 0.31135877, 0.0171975 , '#00ffff'),
        ('W', 'white',     1,   1,   1, 0.6547965 , 0.34516271, 0.01750837, '#ffffff'),
        ('Y', 'yellow',    1,   1,   0, 0.61883073, 0.30895636, 0.00222748, '#ffff00')],
        columns = ['short_name', 'name', 'R', 'G', 'B', 'L', 'M', 'S', 'sRGB_hex'])
    rgb_cube.index = rgb_cube['short_name']
    rgb_cube.index.name = 'key'
    # display(HTML("<h2>RGB Cube</h2>"), rgb_cube)
    
    fig = make_subplots(1,2, 
                        subplot_titles=('RGB Color Space',  'LMS Color Space'),
                        specs=[[{"type": "scene"}, {"type": "scene"}]])

    fig.add_scatter3d(x=rgb_cube['R'], y=rgb_cube['G'], z=rgb_cube['B'], 
                      text=rgb_cube.index, mode="markers+text",
                      showlegend=False,
                      marker=dict(color=rgb_cube['sRGB_hex']),
                      row=1, col=1)

    fig.add_scatter3d(x=rgb_cube['L'], y=rgb_cube['M'], z=rgb_cube['S'], 
                      text=rgb_cube.index, mode="markers+text",
                      showlegend=False,
                      marker=dict(color=rgb_cube['sRGB_hex']),
                      row=1, col=2)

    fig.update_layout(title=title)

    fig.update_layout(dragmode='orbit', 
                      scene = dict(
                        xaxis_title='R',
                        yaxis_title='G',
                        zaxis_title='B'))

    fig.update_layout(scene2 = dict(
                        xaxis_title='L',
                        yaxis_title='M',
                        zaxis_title='S'))
    
    # Add the parallelepiped lines
    df = rgb_cube.loc[['K','B','M','R', 'K', 'G', 'Y', 'W', 'C', 'G', 'Y', 'R', 'M', 'W', 'C', 'B']]    
    fig.add_scatter3d(x=df['R'], y=df['G'], z=df['B'], mode='lines', showlegend=False, row=1, col=1)
    fig.add_scatter3d(x=df['L'], y=df['M'], z=df['S'], mode='lines', showlegend=False, row=1, col=2)

    for line in lines:
        fig.add_scatter3d(x=line['R'], y=line['G'], z=line['B'], showlegend=False, 
                          marker=dict(color=line['sRGB_hex']),
                          row=1, col=1)

        fig.add_scatter3d(x=line['L'], y=line['M'], z=line['S'], showlegend=False, 
                          marker=dict(color=line['sRGB_hex']),
                          row=1, col=2)

    fig.show()

# Confusion lines for Deficiency.PROTAN
protan_confusion_lines = [
    pd.DataFrame.from_records([
        (0.2309, 0.0690, 0.0035, 1.0000, 0.1022, 0.1960, '#fe5a7a'),
        (0.2101, 0.0690, 0.0035, 0.8333, 0.1226, 0.1968, '#eb627a'),
        (0.1893, 0.0690, 0.0035, 0.6667, 0.1430, 0.1977, '#d5697a'),
        (0.1684, 0.0690, 0.0035, 0.5000, 0.1633, 0.1985, '#bb707b'),
        (0.1476, 0.0690, 0.0035, 0.3334, 0.1837, 0.1993, '#9c767b'),
        (0.1268, 0.0690, 0.0035, 0.1667, 0.2041, 0.2002, '#717c7b'),
        (0.1060, 0.0690, 0.0035, 0.0001, 0.2244, 0.2010, '#00827b')],
        columns = ['L', 'M', 'S', 'R', 'G', 'B', 'sRGB_hex']),
    pd.DataFrame.from_records([
        (0.3899, 0.1726, 0.0088, 1.0000, 0.4389, 0.4975, '#feb0bb'),
        (0.3690, 0.1726, 0.0088, 0.8333, 0.4593, 0.4983, '#ebb4bb'),
        (0.3482, 0.1726, 0.0088, 0.6667, 0.4796, 0.4992, '#d5b8bb'),
        (0.3274, 0.1726, 0.0088, 0.5000, 0.5000, 0.5000, '#bbbbbb'),
        (0.3066, 0.1726, 0.0088, 0.3334, 0.5204, 0.5008, '#9cbebb'),
        (0.2858, 0.1726, 0.0088, 0.1667, 0.5407, 0.5017, '#71c2bb'),
        (0.2649, 0.1726, 0.0088, 0.0001, 0.5611, 0.5025, '#00c5bb')],
        columns = ['L', 'M', 'S', 'R', 'G', 'B', 'sRGB_hex']),
    pd.DataFrame.from_records([
        (0.2754, 0.1044, 0.0120, 1.0000, 0.1584, 0.7462, '#fe6ee0'),
        (0.2546, 0.1044, 0.0120, 0.8333, 0.1787, 0.7471, '#eb75e0'),
        (0.2337, 0.1044, 0.0120, 0.6667, 0.1991, 0.7479, '#d57be0'),
        (0.2129, 0.1044, 0.0120, 0.5000, 0.2194, 0.7487, '#bb80e0'),
        (0.1921, 0.1044, 0.0120, 0.3334, 0.2398, 0.7496, '#9c86e0'),
        (0.1713, 0.1044, 0.0120, 0.1667, 0.2602, 0.7504, '#718be0'),
        (0.1505, 0.1044, 0.0120, 0.0001, 0.2805, 0.7513, '#0090e0')],
        columns = ['L', 'M', 'S', 'R', 'G', 'B', 'sRGB_hex']),
    pd.DataFrame.from_records([
        (0.5043, 0.2408, 0.0055, 1.0000, 0.7195, 0.2487, '#fedc88'),
        (0.4835, 0.2408, 0.0055, 0.8333, 0.7398, 0.2496, '#ebdf88'),
        (0.4627, 0.2408, 0.0055, 0.6667, 0.7602, 0.2504, '#d5e189'),
        (0.4419, 0.2408, 0.0055, 0.5000, 0.7805, 0.2513, '#bbe489'),
        (0.4211, 0.2408, 0.0055, 0.3334, 0.8009, 0.2521, '#9ce789'),
        (0.4003, 0.2408, 0.0055, 0.1667, 0.8213, 0.2529, '#71e989'),
        (0.3794, 0.2408, 0.0055, 0.0001, 0.8416, 0.2538, '#00ec89')],
        columns = ['L', 'M', 'S', 'R', 'G', 'B', 'sRGB_hex']),
    pd.DataFrame.from_records([
        (0.5488, 0.2761, 0.0140, 1.0000, 0.7756, 0.7990, '#fee3e6'),
        (0.5280, 0.2761, 0.0140, 0.8333, 0.7959, 0.7998, '#ebe6e7'),
        (0.5072, 0.2761, 0.0140, 0.6667, 0.8163, 0.8007, '#d5e9e7'),
        (0.4864, 0.2761, 0.0140, 0.5000, 0.8367, 0.8015, '#bbebe7'),
        (0.4655, 0.2761, 0.0140, 0.3334, 0.8570, 0.8023, '#9ceee7'),
        (0.4447, 0.2761, 0.0140, 0.1667, 0.8774, 0.8032, '#71f0e7'),
        (0.4239, 0.2761, 0.0140, 0.0001, 0.8977, 0.8040, '#00f3e7')],
        columns = ['L', 'M', 'S', 'R', 'G', 'B', 'sRGB_hex']),
]

show_confusion_lines('Examples of protan confusion lines', protan_confusion_lines)

The colors of some of the 5 lines might be easier to discriminate for some protan, but overall with this point size they should be pretty difficult to differentiate on a monitor screen. Let's also visualize some lines for deuteranopes (parallel to the M axis) and tritanopes (parallel to the S axis).

In [ ]:
# collapse-hide

# Confusion lines for Deficiency.DEUTAN
deutan_confusion_lines = [
    pd.DataFrame.from_records([
        (0.1310, 0.0311, 0.0035, 0.6889, 0.0000, 0.2151, '#d8007f'),
        (0.1310, 0.0400, 0.0035, 0.5741, 0.0470, 0.2116, '#c73d7e'),
        (0.1310, 0.0489, 0.0035, 0.4593, 0.0939, 0.2080, '#b4567d'),
        (0.1310, 0.0578, 0.0035, 0.3445, 0.1409, 0.2045, '#9e687c'),
        (0.1310, 0.0667, 0.0035, 0.2297, 0.1879, 0.2009, '#83787b'),
        (0.1310, 0.0756, 0.0035, 0.1149, 0.2348, 0.1974, '#5f857a'),
        (0.1310, 0.0846, 0.0035, 0.0000, 0.2818, 0.1938, '#009079')],
        columns = ['L', 'M', 'S', 'R', 'G', 'B', 'sRGB_hex']),
    pd.DataFrame.from_records([
        (0.3274, 0.1338, 0.0088, 1.0000, 0.2955, 0.5155, '#fe93be'),
        (0.3274, 0.1467, 0.0088, 0.8333, 0.3636, 0.5103, '#eba2bd'),
        (0.3274, 0.1596, 0.0088, 0.6667, 0.4318, 0.5052, '#d5afbc'),
        (0.3274, 0.1726, 0.0088, 0.5000, 0.5000, 0.5000, '#bbbbbb'),
        (0.3274, 0.1855, 0.0088, 0.3334, 0.5682, 0.4948, '#9cc6ba'),
        (0.3274, 0.1985, 0.0088, 0.1667, 0.6363, 0.4897, '#71d0b9'),
        (0.3274, 0.2114, 0.0088, 0.0001, 0.7045, 0.4845, '#00dab8')],
        columns = ['L', 'M', 'S', 'R', 'G', 'B', 'sRGB_hex']),
    pd.DataFrame.from_records([
        (0.1817, 0.0570, 0.0120, 0.8612, 0.0000, 0.7689, '#ee00e3'),
        (0.1817, 0.0681, 0.0120, 0.7176, 0.0587, 0.7645, '#dc44e2'),
        (0.1817, 0.0792, 0.0120, 0.5741, 0.1174, 0.7600, '#c760e1'),
        (0.1817, 0.0904, 0.0120, 0.4306, 0.1761, 0.7556, '#af74e1'),
        (0.1817, 0.1015, 0.0120, 0.2871, 0.2348, 0.7511, '#9185e0'),
        (0.1817, 0.1127, 0.0120, 0.1436, 0.2935, 0.7467, '#6993e0'),
        (0.1817, 0.1238, 0.0120, 0.0001, 0.3522, 0.7423, '#00a0df')],
        columns = ['L', 'M', 'S', 'R', 'G', 'B', 'sRGB_hex']),
    pd.DataFrame.from_records([
        (0.4731, 0.2214, 0.0055, 1.0000, 0.6477, 0.2577, '#fed28a'),
        (0.4731, 0.2325, 0.0055, 0.8565, 0.7064, 0.2533, '#eeda89'),
        (0.4731, 0.2436, 0.0055, 0.7130, 0.7652, 0.2489, '#dbe288'),
        (0.4731, 0.2548, 0.0055, 0.5694, 0.8239, 0.2444, '#c6ea87'),
        (0.4731, 0.2659, 0.0055, 0.4259, 0.8826, 0.2400, '#aef186'),
        (0.4731, 0.2771, 0.0055, 0.2824, 0.9413, 0.2355, '#90f885'),
        (0.4731, 0.2882, 0.0055, 0.1389, 1.0000, 0.2311, '#68fe84')],
        columns = ['L', 'M', 'S', 'R', 'G', 'B', 'sRGB_hex']),
    pd.DataFrame.from_records([
        (0.5238, 0.3141, 0.0140, 0.3111, 1.0000, 0.7849, '#97fee5'),
        (0.5238, 0.3052, 0.0140, 0.4259, 0.9530, 0.7884, '#aef9e5'),
        (0.5238, 0.2963, 0.0140, 0.5407, 0.9061, 0.7920, '#c2f4e6'),
        (0.5238, 0.2873, 0.0140, 0.6555, 0.8591, 0.7955, '#d3eee6'),
        (0.5238, 0.2784, 0.0140, 0.7703, 0.8121, 0.7991, '#e3e8e6'),
        (0.5238, 0.2695, 0.0140, 0.8851, 0.7652, 0.8026, '#f1e2e7'),
        (0.5238, 0.2606, 0.0140, 1.0000, 0.7182, 0.8062, '#fedce7')],
        columns = ['L', 'M', 'S', 'R', 'G', 'B', 'sRGB_hex']),
]

show_confusion_lines('Examples of deutan confusion lines', deutan_confusion_lines)

In [ ]:
# collapse-hide

# Confusion lines for Deficiency.TRITAN
tritan_confusion_lines = [
    pd.DataFrame.from_records([
        (0.1310, 0.0690, 0.0005, 0.1649, 0.2306, 0.0000, '#708300'),
        (0.1310, 0.0690, 0.0030, 0.1941, 0.2051, 0.1667, '#797d71'),
        (0.1310, 0.0690, 0.0055, 0.2234, 0.1796, 0.3333, '#82759c'),
        (0.1310, 0.0690, 0.0080, 0.2527, 0.1541, 0.5000, '#896dbb'),
        (0.1310, 0.0690, 0.0105, 0.2820, 0.1285, 0.6666, '#9064d5'),
        (0.1310, 0.0690, 0.0130, 0.3113, 0.1030, 0.8333, '#975aeb'),
        (0.1310, 0.0690, 0.0155, 0.3405, 0.0775, 0.9999, '#9d4efe')],
        columns = ['L', 'M', 'S', 'R', 'G', 'B', 'sRGB_hex']),
    pd.DataFrame.from_records([
        (0.3274, 0.1726, 0.0163, 0.5878, 0.4234, 1.0000, '#c9aefe'),
        (0.3274, 0.1726, 0.0138, 0.5586, 0.4489, 0.8333, '#c5b2eb'),
        (0.3274, 0.1726, 0.0113, 0.5293, 0.4745, 0.6667, '#c0b7d5'),
        (0.3274, 0.1726, 0.0088, 0.5000, 0.5000, 0.5000, '#bbbbbb'),
        (0.3274, 0.1726, 0.0062, 0.4707, 0.5255, 0.3334, '#b6bf9c'),
        (0.3274, 0.1726, 0.0037, 0.4414, 0.5510, 0.1667, '#b1c371'),
        (0.3274, 0.1726, 0.0012, 0.4122, 0.5766, 0.0001, '#abc700')],
        columns = ['L', 'M', 'S', 'R', 'G', 'B', 'sRGB_hex']),
    pd.DataFrame.from_records([
        (0.1817, 0.1044, 0.0158, 0.2939, 0.2117, 1.0000, '#937efe'),
        (0.1817, 0.1044, 0.0133, 0.2646, 0.2372, 0.8333, '#8c85eb'),
        (0.1817, 0.1044, 0.0108, 0.2354, 0.2628, 0.6667, '#858cd5'),
        (0.1817, 0.1044, 0.0083, 0.2061, 0.2883, 0.5000, '#7d92bb'),
        (0.1817, 0.1044, 0.0058, 0.1768, 0.3138, 0.3334, '#74979c'),
        (0.1817, 0.1044, 0.0032, 0.1475, 0.3393, 0.1667, '#6b9d71'),
        (0.1817, 0.1044, 0.0007, 0.1182, 0.3649, 0.0001, '#60a200')],
        columns = ['L', 'M', 'S', 'R', 'G', 'B', 'sRGB_hex']),
    pd.DataFrame.from_records([
        (0.4731, 0.2408, 0.0168, 0.8818, 0.6351, 1.0000, '#f1d0fe'),
        (0.4731, 0.2408, 0.0143, 0.8525, 0.6606, 0.8333, '#edd4eb'),
        (0.4731, 0.2408, 0.0118, 0.8232, 0.6862, 0.6667, '#ead7d5'),
        (0.4731, 0.2408, 0.0093, 0.7939, 0.7117, 0.5000, '#e6dbbb'),
        (0.4731, 0.2408, 0.0067, 0.7646, 0.7372, 0.3334, '#e2de9c'),
        (0.4731, 0.2408, 0.0042, 0.7354, 0.7628, 0.1667, '#dee271'),
        (0.4731, 0.2408, 0.0017, 0.7061, 0.7883, 0.0001, '#dae500')],
        columns = ['L', 'M', 'S', 'R', 'G', 'B', 'sRGB_hex']),
    pd.DataFrame.from_records([
        (0.5238, 0.2761, 0.0170, 0.8351, 0.7694, 1.0000, '#ebe3fe'),
        (0.5238, 0.2761, 0.0145, 0.8059, 0.7949, 0.8333, '#e7e6eb'),
        (0.5238, 0.2761, 0.0120, 0.7766, 0.8204, 0.6667, '#e4e9d5'),
        (0.5238, 0.2761, 0.0095, 0.7473, 0.8459, 0.5000, '#e0ecbb'),
        (0.5238, 0.2761, 0.0070, 0.7180, 0.8715, 0.3334, '#dcf09c'),
        (0.5238, 0.2761, 0.0045, 0.6887, 0.8970, 0.1667, '#d8f371'),
        (0.5238, 0.2761, 0.0020, 0.6595, 0.9225, 0.0001, '#d4f600')],
        columns = ['L', 'M', 'S', 'R', 'G', 'B', 'sRGB_hex']),
]

show_confusion_lines('Examples of tritan confusion lines', tritan_confusion_lines)

At this stage if you are affected by a color vision deficiency you might already have a more precise idea of what kind of dichromacy better models your color vision. For me the colors of the protan confusion lines look pretty similar for such a small point size, but I can better distinguish the deutan lines and very well the tritan ones.

So now that we can compute these confusion lines for each kind of dichromacy, can we simulate how a person with a CVD sees? Since it's related to perception it's impossible to know how another person "feels" the colors, but to understand the loss of color sensitivity we can transform images so that all the colors on each confusion line projects to a single one, essentially making the color space 2D. If the models are correct then the transformed image should still appear identical to a dichromat, and a trichromat will get a sense of the color differences that get lost for the dichromat.

The remaining question is what color should we pick to represent each confusion line? That will be the subject of the [Simulating dichromacy in LMS](#Simulating-dichromacy-in-LMS) section.

Finally once the LMS colors are projected along the confusion lines, we can transform them back to the linearRGB and then sRGB color spaces to get the final simulated image. The overall process is summarized in the diagram below, and we're going to dive into each part in the next sections.

![image](simulation_images/CVD_Simulation_Pipeline.svg "CVS Simulation Pipeline")

# From sRGB to LMS

In this section we're going to review the steps to go to that nice LMS color space. Several models have been proposed in the litterature, so there is not just one way to do it.

## From sRGB to linearRGB

Most digital images are encoded with 3 values per pixel: red, green and blue. But what is often neglected is that this "RGB" usually corresponds to the sRGB standard that precisely define how these values should be interpreted. sRGB is not a linear space because it tries to compensate the non-linear gamma factor applied by monitors, so that the image captured by a digital camera and then rendered on a monitor will look like the original scene.

We'll call the RGB color space after removing that non-linearity the *linearRGB* color space. The transformation roughly corresponds to applying a gamma exponent of 2.4 on each channel like a typical monitor would do, with a special linear slope for very small values. So unlike all the other color transformations of that notebook it is non-linear and cannot be expressed as a matrix.

The good news is that sRGB is a well-defined standard now, so we just need to implement the function given by the standard. For more details see [the wikipedia page](https://en.wikipedia.org/wiki/SRGB).

Since both Brettel 1997 and Viénot 1999 were published before the sRGB standard became mainstream, they had to calibrate their old CRT monitor and publish their own model in the paper. So their matrices must be adapted to use the new sRGB model. Even worse, even if the Viénot 1999 paper explicitely mentions the required non-linear transform for their CRT monitor, the most widely replicated matlab code from {% cite fidaner_analysis_2005 %} does not take it into account and directly treat the input image as linear RGB. As a consequence this is also the case of most of the source code listed on {% cite daltonize_org %}.

### Does it matter? Yes!

As you can see in the image below forgetting the sRGB -> linearRGB transform makes a big difference and it can invalidate perceptual experiments. The first row has 4 sRGB colors, and the second row shows the same colors wrongly interpreted as linear RGB.

In [ ]:
# collapse-hide
im_sRGB = np.array([[[127,0,0], [64,128,0], [0, 0, 192], [64, 64, 192]]], dtype=float) / 255.0
px.imshow(np.vstack([im_sRGB, # First row is the sRGB content
                     convert.linearRGB_from_sRGB(im_sRGB)]), 
          title='sRGB(first row) vs linear RGB (second row)').update_layout(hide_image_axes)

And the impact is also significant on the final simulation, here is a comparison of Vienot 1999 implemented with and without the proper sRGB decoding / encoding. Note how a bunch of colors appear too dark in the simulation without sRGB.

<!-- Image tags have to be unindented for nbdev to catch them :-( -->
<table>
    <tr><th>Original</th><th> Viénot 1999</th><th>Viénot 1999 without sRGB</th></tr>
    <tr><td>
<img src="simulation_images/rgbspan.png"></td><td>
<img src="simulation_images/rgbspan_protan_vienot1999.png"></td><td>
<img src="simulation_images/rgbspan_protan_vienot1999_missing_srgb.png"></td></tr>
</table>

## From linearRGB to CIE XYZ

So far we only mentioned the LMS color space, saying that we'd just go from linearRGB to LMS. But to understand the litterature around that transform it is necessary to add an intermediate step which is the CIE XYZ color space. The reason is historical, this color space was standardized in 1931 and it has been used as the main space for color science research since. The main reason is that it can (almost) represent the full range of colors that can be perceived by an average human observer with just 3 values, is device-independent, and has a well-defined relationship w.r.t the input light spectrum.

> Note: dichromats confusion lines are sometimes shown in the CIE XYZ color space (e.g. on [color-blindness.com](http://www.color-blindness.com/2009/01/19/colorblind-colors-of-confusion/)), usually drawing the lines on a so-called chromaticity diagram. These diagram are a 2d projection of the CIE XYZ color space for a constant maximal luminance. I always got confused about those because I could clearly see differences along these lines, especially when they transition from green to red. The reason is that the confusion lines are actually 3d in CIE XYZ and the luminance needs to get adjusted as the hue/chroma changes along the line to actually get confused by a dichromat.

The [sRGB wikipedia page](https://en.wikipedia.org/wiki/SRGB#The_reverse_transformation_(sRGB_to_CIE_XYZ) gives the `XYZ_from_linearRGB` matrix defined by the standard itself. As we noted Viénot 1999 used a different matrix resulting from a generic CRT monitor. That matrix was kept in {% cite fidaner_analysis_2005 %} and is thus still widespread today.

However as {% cite simon_liedtke_using_2016 %} noted, we should now use the modern sRGB-induced transform for LCD monitors. This is also the model correctly used by {% cite dietrich_daltonize_2020 %} and {% cite jim_ixora_2021 %}.

In [ ]:
# collapse-hide
printMatrix('Vienot1999 XYZ_from_linearRGB', convert.LMSModel_Vienot1999_SmithPokorny75().XYZ_from_linearRGB*255.0); print()
printMatrix('sRGB XYZ_from_linearRGB', convert.LMSModel_sRGB_SmithPokorny75().XYZ_from_linearRGB*255)

## From CIE XYZ to LMS

That conversion is less clear as many more transforms have been proposed, with different motivations. The most relevant models for CVD simulations are based on color matching experiments with people who happen to have one normal trichromat eye and one dichromat eye (unilateral dichromats).

The most widely used conversion for CVD simulation is still {% cite smith_spectral_1975 %}, which is the one used by Brettel 1997 and Viénot 1999. The earlier seminal work of {% cite meyer_color_defective_1988 %} used another transform from {% cite estevez1981fundamental %} but it already mentioned {% cite smith_spectral_1975 %} as being a good candidate. Another popular reference model in the litterature is the Hunt-Pointer-Esterez one {% cite estevez1981fundamental, hunt_reproduction_2004 %}. It is used a lot for chromatic adaptation (predicting how a perceived color will change with the illumination conditions), but less for CVD simulation experiments.

{% cite stockman_spectral_2000 %} made a thorough comparison of the models, and they also propose a new matrix (for a 10 degree standard observer). But overall they are in good agreement with {% cite smith_spectral_1975 %} once adjusted for a 2 degree observer, with larger differences compared to Hunt-Pointer-Esterez. All 3 are probably reasonable options, but since the CVD simulation evaluations were most often made and evaluated with Smith & Pokorny it's probably still the most solid choice, despite its age. It is also interesting to note that both Pokorny and Viénot continue to promote that transform in more recent papers {% cite vienot_colorimetry_2013 %} {% cite smith_color_2003 %}, and other recent papers in the color science research litterature do the same {% cite machado_physiologically_based_2009 %} {% cite simon_liedtke_using_2016 %}.

Most opensource code uses the Smith and Pokorny since they did not modify the original code, but {% cite jim_ixora_2021 %} uses the Hunt-Pointer-Estevez matrix and {% cite dietrich_daltonize_2020 %} uses the sharpened transformation matrix of CIECAM02. These two matrices are on the Wikipedia page and look attractive as they are more recent. But at least the "sharpened" matrices variants are not meant to match the physiological cone responses but to perform hue-preserving chromatic adaptation (see the discussion p181 of {% cite fairchild2013color %}). So I would argue that we should keep using Smith & Pokorny transform until a proper validation of newer matrices gets done for CVD simulation.

> Note: the XYZ<->LMS matrices of the different models can look very different, but the main reason is the normalization factor. There is no unique "scale" for each axis, so each model picked a convention. For example Smith & Pokorny made it so L+S=Y and S/Y=1 at the 400nm wavelengths. For Hunt-Pointer-Esterez they normalize it so that L=M=S for the equal-energy illuminant E (X=Y=Z=1/3), which means that the sum of each rows equals to 1. One reason for this scale variance is that the human vision system can actually change it independently for each axis depending on the illumination conditions. This process is called _chromatic adaptation_ and explains why we still perceive a white sheet of paper as white under an incandescent indoor lighting that makes it actually yellow-ish. It's a bit similar to the white balance correction of digital cameras. These normalizations do not really matter for CVD simulation since the angles between the axis do not change with it and the scale factor will get removed when we go back to RGB.

Below are some example of those `LMS_from_XYZ` matrices.

In [ ]:
# collapse-hide
printMatrix('Smith & Pokorny (Viénot) LMS_from_XYZ', convert.LMSModel_Vienot1999_SmithPokorny75().LMS_from_XYZ); print()
printMatrix('Hunt-Pointer-Estevez LMS_from_XYZ', convert.LMSModel_sRGB_HuntPointerEstevez().LMS_from_XYZ); print()
printMatrix('Stockman-Sharpe LMS_from_XYZ', convert.LMSModel_sRGB_StockmanSharpe2000().LMS_from_XYZ); print()

## Putting it together

Since both `XYZ_from_linearRGB` and `LMS_from_XYZ` are linear transforms, we can just combine the two matrices to get a final `LMS_from_linearRGB = LMS_from_XYZ . XYZ_from_linearRGB` transform. Here are some of the models implemented in DaltonLens, scaled by 255 for an easier comparison with existing opensource code:

In [ ]:
# collapse-hide
printMatrix('Viénot 1999\nLMS_from_linearRGB', convert.LMSModel_Vienot1999_SmithPokorny75().LMS_from_linearRGB*255.0); print()
printMatrix('sRGB Viénot\nLMS_from_linearRGB', convert.LMSModel_sRGB_SmithPokorny75().LMS_from_linearRGB*255.0); print()
printMatrix('Hunt-Pointer-Esterez\nLMS_from_linearRGB', convert.LMSModel_sRGB_HuntPointerEstevez().LMS_from_linearRGB*255.0); print()
printMatrix('Stockman & Sharpe\nLMS_from_linearRGB', convert.LMSModel_sRGB_StockmanSharpe2000().LMS_from_linearRGB*255.0); print()

# Simulating dichromacy in LMS (Brettel & Mollon 1997)

Let's rebuild the seminal paper of {% cite brettel_computerized_1997 %} that built a precise model based on perceptual experiments. {% cite vienot_digital_1999 %} then simplified it for its use with digital monitors, reducing the whole pipeline to a single matrix multiplication, and we'll see it right after.

## Projection planes in LMS

Now that we are in a color space that better matches the human physiology, the question becomes how to transform the LMS coordinates to simulate what a color blind person perceives. As discussed in the FIXME: introduction, we want all the colors along the confusion lines to project to the same one, so that a dichromat won't see a difference in the transformed image, but a trichromat will see how much of the color information is lost by the dichromat.

As {% cite jim_ixora_2021 %} points out, a first idea is to just set the deficient axis to 0. But that won't work for several reasons:

1) The new point with L=0, S=0 or M=0 is likely to be outside of the sRGB gamut (the parallelepiped we visualized before in LMS), so most of these transformed color would not be valid once transformed back to sRGB.

2) Experiments on unilateral dichromats (people who have one normal trichromat eye and one dichromat eye) show that they perceive some colors similarly with both eyes. Unsurprisingly among these invariant stimulus we find shades of gray (black to white). Then the similar colors for the two eyes are observed along the 475nm (blue-greenish) and 575nm (yellow) axes for protans and deutans, and along the 485nm (cyan) and 660nm (red) axes for tritans. {% cite meyer_color_defective_1988 %} has solid references for some of these experiments, made in the 1950s. Intuitively it makes sense that the dichromat eye of a protan or deutan does a good job on colors that only differ by the amount of blue (S cone). Indeed in LMS going from black to blue or white to yellow is just a change in the S coordinate.

Based on that that data it is possible to design a projection that respect these constraints. {% cite brettel_computerized_1997 %} proposed the first modern algorithm to compute that. It consists in creating two half-planes around the Black-White diagonal. For protanopes and deuteranopes the wings go through the LMS coordinates of the 475nm and 575nm wavelengths, and for tritanopes towards the coordinates of the 485nm and 660nm wavelenghts. By construction colors that lie on these invariant axes are thus preserved.

While black just has a (0,0,0) coordinate in LMS, "white" needs a more careful definition. In {% cite brettel_computerized_1997 %} they take the white point of the equal-energy illuminant as the neutral axis. That illuminant (called `E` in [CIE XYZ terminology](https://en.wikipedia.org/wiki/Standard_illuminant#Illuminant_E)) has the same amount of energy on all the visible light wavelenghts, so it presents a perfectly even color. It differs a bit from the "white" of a sRGB monitor which is the white point of the [D65 illuminant](https://en.wikipedia.org/wiki/Illuminant_D65), which roughly corresponds to the stimulus we'd perceive as white under average outdoor daylight. So to stay close to the best theoretical value we'll start by considering Black to E as the neutral axis that dichromats perceive similarly as trichromats.

Let's first visualize how these planes look like in 3D for protans/deutans (they share the same plane) and for tritans. Note that the colors corresponding to pure 475nm, 485nm, 575nm and 660nm cannot be represented on a sRGB monitor, so I use their closest approximation in the plot by desaturating them (adding "white" to them until they fit in the gamut).

In [ ]:
# collapse-hide

lms_model = convert.LMSModel_sRGB_SmithPokorny75()
# lms_model = convert.LMSModel_Vienot1999_SmithPokorny75()
# lms_model = convert.LMSModel_sRGB_StockmanSharpe2000()

import colour # from pip install colour-science
from colour import MSDS_CMFS
cmfs = MSDS_CMFS['CIE 1931 2 Degree Standard Observer']
xyz_475 = colour.wavelength_to_XYZ(475, cmfs)
xyz_575 = colour.wavelength_to_XYZ(575, cmfs)
xyz_485 = colour.wavelength_to_XYZ(485, cmfs)
xyz_660 = colour.wavelength_to_XYZ(660, cmfs)

# From https://www.johndcook.com/wavelength_to_RGB.html
# Approximate since the XYZ coordinates are not actually in the RGB gamut
# The correspondence has to be done projecting in the direction of the
# whitepoint until reaching the gamut.
# http://www.fourmilab.ch/documents/specrend/

lms_475 = lms_model.LMS_from_XYZ @ xyz_475
lms_575 = lms_model.LMS_from_XYZ @ xyz_575
lms_485 = lms_model.LMS_from_XYZ @ xyz_485
lms_660 = lms_model.LMS_from_XYZ @ xyz_660

# The equal-energy white point. By construction of CIE XYZ
# it has X=Y=Z. The normalization does not matter to define
# the diagonal direction, picking 0.8 to make it close to
# sRGB white.
xyz_E = [0.8, 0.8, 0.8]
rgb_E = lms_model.linearRGB_from_XYZ @ xyz_E
srgb_E = convert.sRGB_from_linearRGB([rgb_E])
lms_E = lms_model.LMS_from_XYZ @ xyz_E

rgb_wavelengths = convert.apply_color_matrix(np.array([xyz_475, xyz_575, xyz_485, xyz_660]), lms_model.linearRGB_from_XYZ)
rgb_wavelengths = convert.desaturate_linearRGB_to_fit_in_gamut(rgb_wavelengths)
srgb_wavelengths = convert.sRGB_from_linearRGB(rgb_wavelengths)

rgb_475, rgb_575, rgb_485, rgb_660 = srgb_wavelengths

rgb_gamut = [
    (np.array([0.0, 0.0, 0.0]), 'K', 'black'),
    (np.array([0.0, 0.0, 1.0]), 'B', 'blue'),
    (np.array([1.0, 0.0, 1.0]), 'M', 'magenta'),
    (np.array([1.0, 0.0, 0.0]), 'R', 'red'),    
    (np.array([0.0, 1.0, 0.0]), 'G', 'green'),
    (np.array([0.0, 1.0, 1.0]), 'C', 'cyan'),
    (np.array([1.0, 1.0, 1.0]), 'W', 'white'),    
    (np.array([1.0, 1.0, 0.0]), 'Y', 'yellow'),
]

def dataframe_row(rgb, lms, short_name:str, name: str):
    int_rgb255 = (rgb*255.0).astype(np.uint8)
    return [name, short_name, *rgb, *lms, f'#{int_rgb255[0]:02x}{int_rgb255[1]:02x}{int_rgb255[2]:02x}']

def lms_of_key(df, key):
    return np.array(df.loc[key, ['L','M','S']], dtype=float)

# RGB parallelepiped gamut in LMS
df_rows = [dataframe_row(row[0], lms_model.LMS_from_linearRGB @ row[0], row[1], row[2]) for row in rgb_gamut]
df_rows.append(dataframe_row(rgb_E, lms_E, 'E', 'Equal-Energy'))
df_rows.append(dataframe_row(rgb_475, lms_475, '475nm', '475nm'))
df_rows.append(dataframe_row(rgb_575, lms_575, '575nm', '575nm'))
df_rows.append(dataframe_row(rgb_485, lms_485, '485nm', '485nm'))
df_rows.append(dataframe_row(rgb_660, lms_660, '660nm', '660nm'))
KBMRGCWY = pd.DataFrame(df_rows, columns=['name', 'short_name', 'R','G','B','L','M','S','colorhex'])
KBMRGCWY.set_index('short_name', inplace=True)
KBMRGCWY['short_name'] = KBMRGCWY.index

KBMRGCWY_red_green = KBMRGCWY[0:-2] # drop 485 and 660nm
KBMRGCWY_tritan = KBMRGCWY.drop(KBMRGCWY.index[-4:-2]) # drop 475 and 575nm    
   
figs = [
    px.scatter_3d(KBMRGCWY_red_green, x='L', y='M', z='S', 
                  color='colorhex', color_discrete_map='identity', text='short_name',
                  width=800, height=800, 
                  title='Protan and Deutan Projection Half-Planes according to (Brettel & Mollon, 1997)'),
    
    px.scatter_3d(KBMRGCWY_tritan, x='L', y='M', z='S', 
                  color='colorhex', color_discrete_map='identity', text='short_name',
                  width=800, height=800,
                  title='Tritan Projection Half-Planes according to (Brettel & Mollon, 1997)')
]
    

# Add the parallelepiped lines
df = KBMRGCWY.loc[['K','B','M','R', 'K', 'G', 'Y', 'W', 'C', 'G', 'Y', 'R', 'M', 'W', 'C', 'B']]
for fig in figs:
    fig.add_scatter3d(x=df['L'], y=df['M'], z=df['S'], mode='lines', showlegend=False)

# Add the black-E diagonal, dashed
df = KBMRGCWY.loc[['K','E']]
for fig in figs:
    fig.add_scatter3d(x=df['L'], y=df['M'], z=df['S'], mode='lines', showlegend=False, line=dict(width=8, dash='dash', color='gray'), row=1, col=1)

def add_half_plane(fig, df, wavelength_key, col, U_norm, V_norm, delaunayaxis='x'):
    # The plane is defined by two vectors starting from the origin
    # U is the black-white diagonal, and V is 
    U =  lms_of_key(df, 'E') - lms_of_key(df, 'K')
    U = U_norm * (U / np.linalg.norm(U))
    V =  lms_of_key(df, wavelength_key) - lms_of_key(df, 'K')
    N = np.cross(U,V)
    V = np.cross(N,U)
    V = V_norm * (V / np.linalg.norm(V))
    u,v = np.mgrid[0:1:50j, 0:1:50j]
    xyz = u[...,np.newaxis]*U + v[...,np.newaxis]*V
    x = xyz[...,0].flatten()
    y = xyz[...,1].flatten()
    z = xyz[...,2].flatten()
    surfacecolor = convert.apply_color_matrix(xyz, lms_model.linearRGB_from_LMS)
    surfacecolor = convert.desaturate_linearRGB_to_fit_in_gamut(surfacecolor)
    surfacecolor = surfacecolor.reshape(-1,3)
    #display(surfacecolor)
    surfacecolor = convert.sRGB_from_linearRGB(surfacecolor)
    fig.add_mesh3d(z=z, x=x, y=y, opacity=1.0, delaunayaxis=delaunayaxis, alphahull=-1, vertexcolor=surfacecolor, 
                   lighting=dict(ambient=1.0,specular=0.0,diffuse=0.0), 
                   showlegend=False)
    
df = KBMRGCWY_red_green
add_half_plane(figs[0], df, '475nm', 1, np.linalg.norm(lms_of_key(df, 'E'))*1.5, lms_of_key(df, 'B')[2]*3)
add_half_plane(figs[0], df, '575nm', 1, np.linalg.norm(lms_of_key(df, 'E'))*1.5, lms_of_key(df, 'B')[2]*3)

df = KBMRGCWY_tritan
add_half_plane(figs[1], df, '485nm', 2, np.linalg.norm(lms_of_key(df, 'E'))*1.5, np.linalg.norm(lms_of_key(df, 'B'))*2, 'y')
add_half_plane(figs[1], df, '660nm', 2, np.linalg.norm(lms_of_key(df, 'E'))*1.5, np.linalg.norm(lms_of_key(df, 'R'))*0.5, 'y')

camera = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=-1.75, y=-1.75, z=0.75),
)
for fig in figs:
    fig.update_layout(scene_camera=camera, dragmode='orbit')
    fig.show()

If the model were perfect then these planes would show the entire set of colors that can be perceived by a dichromat. The idea of {% cite brettel_computerized_1997 %} is then to project every LMS color into these planes along the dichromat missing axis, picking the closest of the 2 planes. But from these 3d plots we observe two things:

- For protans and deutans the two wings are almost co-planar, and the plane is very close to passing through the white, blue and yellow limits of the sRGB parallelepiped. So {% cite vienot_digital_1999 %} proposed to simplify the algorithm by using a single diagonal plane black-white-blue-yellow. As we'll see next this makes it possible to include the projection as a single 3x3 matrix and keep a fully linear pipeline.

- {% cite vienot_digital_1999 %} did not address the tritanope case though, and we can see that such an approximation would not be as good. Several opensource implementations of the paper just use the same black-white-blue-yellow diagonal plane as the protan/deutan one, which is completely wrong, as {% cite jim_ixora_2021 %} mentions. A better single plane approximation would be black-white-red-cyan, but it's still a rough approximation and unclear how accurate this is as evaluation studies are rare for tritanopes.

Let's visualize how the Viénot simplication looks like and derive their full algorithm.

## The simplification of (Viénot, Brettel and Mollon 1999) for protanopes and deuteranopes

In {% cite vienot_digital_1999 %} a single plane black-white-blue-yellow is created and all the colors will get projected onto it. Let's visualize that plane in 3D.

In [ ]:
# collapse-hide

# RGB parallelepiped gamut in LMS
fig = px.scatter_3d(KBMRGCWY.loc['K':'Y'], x='L', y='M', z='S', 
                  color='colorhex', color_discrete_map='identity', text='short_name',
                  width=800, height=800, 
                  title='Protan and Deutan Projection Plane according to (Viénot & Brettel & Mollon, 1999)')

# Add the parallelepiped lines
df = KBMRGCWY.loc[['K','B','M','R', 'K', 'G', 'Y', 'W', 'C', 'G', 'Y', 'R', 'M', 'W', 'C', 'B']]
fig.add_scatter3d(x=df['L'], y=df['M'], z=df['S'], mode='lines', showlegend=False)

# Add the black-white diagonal, dashed
df = KBMRGCWY.loc[['K','W']]
fig.add_scatter3d(x=df['L'], y=df['M'], z=df['S'], mode='lines', showlegend=False, line=dict(width=8, dash='dash', color='gray'), row=1, col=1)

def add_black_white_yellow_blue_plane(fig, df):
    # The plane is defined by two vectors starting from the origin
    # U is the black-white diagonal, and V is 
    U =  lms_of_key(df, 'Y') - lms_of_key(df, 'K')
    V =  lms_of_key(df, 'B') - lms_of_key(df, 'K')
    u,v = np.mgrid[0:1:50j, 0:1:50j]
    xyz = u[...,np.newaxis]*U + v[...,np.newaxis]*V
    x = xyz[...,0].flatten()
    y = xyz[...,1].flatten()
    z = xyz[...,2].flatten()
    surfacecolor = convert.apply_color_matrix(xyz, lms_model.linearRGB_from_LMS)
    surfacecolor = convert.desaturate_linearRGB_to_fit_in_gamut(surfacecolor)
    surfacecolor = surfacecolor.reshape(-1,3)
    surfacecolor = convert.sRGB_from_linearRGB(surfacecolor)
    fig.add_mesh3d(z=z, x=x, y=y, opacity=1.0, delaunayaxis='x', alphahull=-1, vertexcolor=surfacecolor, 
                   lighting=dict(ambient=1.0,specular=0.0,diffuse=0.0), 
                   showlegend=False)

df = KBMRGCWY
add_black_white_yellow_blue_plane(fig, df)

camera = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=0, y=-1.75, z=0.75),
)
fig.update_layout(scene_camera=camera, dragmode='orbit')
fig.show()

This plane represents all the colors that can be output by the simulation algorithm. If the theory is right, protans and deutans dichromats can always find a color on that plane that they perceive as similary to any color of the full gamut. Now let's do the math to actually compute the projection of any color into that plane and get the final pipeline.

### Plane projection as a 3x3 matrix

Let $p = (p_l, p_m, p_s)$ be the LMS color to transform, and $n$ the normal of the plane. Our planes always go through the origin `K=(0,0,0)`(black), so the plane equations are just given by their normal $n = (n_l, n_m, n_s)$ as $n.p = n_l.p_l + n_m.p_m + n_s.p_s = 0$. Let's call $p' = (p_l', p_m', p_s')$ the transformed point, projected on the plane. Since only the coordinate along the dichromacy axis changes, we just need to compute the missing axis coordinate and keep the other two unchanged. For example for a protan, we have $p_m' = p_m$ and $p_s' = p_s$, and plugging them into the plane equation we can calculate $p_l'$:

$$
n_l . p_l' + n_m . p_m' + n_s . p_s' = 0 \\
p_l'  = -\frac{n_m}{n_l} . p_m - \frac{n_s}{n_l} . p_s
$$

So we just need the normal of the plane, which is given by the cross-product of two vectors on the plane. Here are the normal of the protan/deutan black-white-blue-yellow plane and the resulting $-\frac{n_m}{n_l}$ and $- \frac{n_s}{n_l}$ factors for a protan:

In [ ]:
v_blue = lms_of_key(KBMRGCWY, 'B') # - K which is ommitted since it's zero
v_yellow = lms_of_key(KBMRGCWY, 'Y')  # - K which is ommitted since it's zero
n = np.cross(v_yellow, v_blue)
print ('Normal (protan & deutan) = ', n)

print ('n_m / n_l = ', n[1]/n[0])
print ('n_s / n_l = ', n[2]/n[0])

These factors can be put into a matrix $H$ so that $p' = H . p$:

In [ ]:
H_protan = np.array([
    [0., -n[1]/n[0], -n[2]/n[0]],
    [0, 1, 0],
    [0, 0, 1] 
])

print()
printMatrix('Projection matrix for protan', H_protan)

Similarly we can get the deutan projection matrix, with $p_l' = p_l$, $p_s' = p_s$ and $p_m' = -\frac{n_l}{n_m} . p_l - \frac{n_s}{n_m} . p_s$ this time:

In [ ]:
H_deutan = np.array([
    [1, 0, 0],
    [-n[0]/n[1], 0, -n[2]/n[1]],
    [0, 0, 1] 
])

print()
printMatrix('Projection matrix for deutan', H_deutan)

> Note: these matrices don't exactly match the Viénot 1999 paper because we used the sRGB version of the model (`convert.LMSModel_sRGB_SmithPokorny75`). They would match if we used `lms_model = LMSModel_Vienot1999_SmithPokorny75` instead at the beginning of the code section.

### Putting it together

Since the projection is just a 3x3 linear matrix, we can combine all the matrices to transform a linear RGB point into its dichromat counterpart and then go back to linear RGB. We also need to add the non-linear from/to conversion between sRGB and linearRGB to get the final Viénot 1999 pipeline:

`srgb_protan = sRGB_from_linearRGB(linearRGB_from_LMS . H_protan . LMS_from_linearRGB . linearRGB_from_sRGB(srgb))`

`srgb_deutan = sRGB_from_linearRGB(linearRGB_from_LMS . H_deutan . LMS_from_linearRGB . linearRGB_from_sRGB(srgb))`

We are almost done, we just need to deal with a last hiccup. Some LMS coordinates of the sRGB gamut might fall outside of the parallelepiped once projected onto the plane. For example if we apply the pipeline to the pure green color `[0,1,0]` we get:

In [ ]:
green_protan = lms_model.linearRGB_from_LMS @ H_protan @ lms_model.LMS_from_linearRGB @ np.array([0.0, 1.0, 0.0])
green_protan

The blue coordinate becomes negative because green falls slighly outside of the parallelepiped once projected along the L axis onto the black-white-blue-yellow plane. Here the error is very small so it's ok to just clip the final values to [0,1]. This is the approach taken by {% cite simon_liedtke_using_2016 %}.

Another way if we use models where we can have larger errors is to desaturate the color by adding a white component until all the coordinates become positive. This is what we did for the `rgb_475`, `rgb_575`, `rgb_485` and `rgb_660` visualization of the wavelengths. Just clipping was totally wrong for those as some had large negative values. See [this page of the fourmilab](https://www.fourmilab.ch/documents/specrend/) for more details, it's implemented as `convert.desaturate_linearRGB_to_fit_in_gamut` in DaltonLens.

Now it's time to see what kind of results we can get!

In [ ]:
# collapse-hide

def simulate_dichromacy_vienot(im, projection_matrix):
    color_mat = lms_model.linearRGB_from_LMS @ projection_matrix @ lms_model.LMS_from_linearRGB
    im_linear_rgb = convert.linearRGB_from_sRGB(convert.as_float32(im))
    im_protan_linear_rgb = convert.apply_color_matrix(im_linear_rgb, color_mat)
    return convert.as_uint8(convert.sRGB_from_linearRGB(im_protan_linear_rgb))

import urllib.request
import cv2

pngdata = urllib.request.urlopen('https://upload.wikimedia.org/wikipedia/commons/d/d7/Byrcolorwheel.png').read()
# pngdata = urllib.request.urlopen('http://www.vischeck.com/images//poppies.jpg').read()
im = cv2.imdecode(np.frombuffer(pngdata, dtype=np.uint8), cv2.IMREAD_COLOR)
im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB) # OpenCV flips the channels by default.

im_protan = simulate_dichromacy_vienot(im, H_protan)
im_deutan = simulate_dichromacy_vienot(im, H_deutan)
im_original = im.copy()
cv2.putText(im_original, 'Original',        (157,190), cv2.FONT_HERSHEY_COMPLEX, 0.5, (64,64,64), 1, cv2.LINE_AA)
cv2.putText(im_protan,   'Protan (Vienot)', (125,190), cv2.FONT_HERSHEY_COMPLEX, 0.5, (64,64,64), 1, cv2.LINE_AA)
cv2.putText(im_deutan,   'Deutan (Vienot)', (125,190), cv2.FONT_HERSHEY_COMPLEX, 0.5, (64,64,64), 1, cv2.LINE_AA)
px.imshow(np.hstack([im_original, im_protan, im_deutan])).update_xaxes(showticklabels=False).update_yaxes(showticklabels=False)

### What about tritanopes?

If we want a one-plane solution for tritanopes we can use the same method but using the black-white-red-cyan plane instead of the black-white-blue-yellow plane we used for protanopes and deuteranopes. As we observed before it's not a very nice fit, but let's try it anyway and we'll compare that with the Bretell method next.

In [ ]:
v_red = lms_of_key(KBMRGCWY, 'R') # - K which is ommitted since it's zero
v_cyan = lms_of_key(KBMRGCWY, 'C')  # - K which is ommitted since it's zero
n = np.cross(v_cyan, v_red)
print ('Normal (tritan) = ', n)

H_tritan = np.array([
    [1, 0, 0],
    [0, 1, 0],
    [-n[0]/n[2], -n[1]/n[2], 0],
])

print()
printMatrix('Projection matrix for tritan', H_tritan)

im_tritan = simulate_dichromacy_vienot(im, H_tritan)
cv2.putText(im_tritan,   'Tritan (Vienot)', (125,190), cv2.FONT_HERSHEY_COMPLEX, 0.5, (64,64,64), 1, cv2.LINE_AA)
px.imshow(np.hstack([im_original, im_tritan])).update_xaxes(showticklabels=False).update_yaxes(showticklabels=False)

## Comparing Viénot 1999 and Brettel 1997

Let's implement {% cite brettel_computerized_1997 %} so we can compare and see whether the extra complexity is worth it. The implementation is very similar to the Viénot approach, but instead of projecting on just one plane we have to project to one of the 2 planes. The plane on which to project depends of the LMS color. The paper directly gives a simple criteria to decide, but it was not so obvious to me so I re-did the math. 

Let's take the example of tritanopia to reason about this:

- If the LMS color is on the "same side" as the 485nm point then we should project on that half-plane, otherwise on the 660nm half-plane.

- The separation is determined by the diagonal $K-E$

- The projection is along the S axis, so we need to determine if the point is on the same side of the plane oriented along the S axis that goes through the K-E diagonal (black-equal energy). A third point on that plane is for example $K+(0,0,1) = (0,0,1)$ since $K$ is the origin.

- Now to determine the side of any point $P = (P_l,P_m,P_s)$ w.r.t to a plane, we can check the sign of the dot-product of a vector going from the plane to the point with the plane normal.

- The plane normal is given by the cross-product of two vectors on the plane: $$n = (E-K) \times ((0,0,1)-K) = E \times (0,0,1) = (E_m,-E_l,0)$$

- So the dot product becomes: $P . n = P_l.E_m - P_m.E_l$. And this is positive if $P_l.E_m > P_m.E_l$. The Brettel paper gives the equivalent condition $\frac{P_l}{P_m} > \frac{E_l}{E_m}$ since all the quantities are positive.

The extra matrix multiplication is not really significant with the processing power we have in 2021, but it never hurts to keep things simpler. Let's start with the tritanopes for once since those are the most likely to have a significant difference. We first need to compute the 2 projection planes.

In [ ]:
v_E = lms_of_key(KBMRGCWY, 'E') # - K which is ommitted since it's zero
v_485 = lms_of_key(KBMRGCWY, '485nm') # - K which is ommitted since it's zero
v_660 = lms_of_key(KBMRGCWY, '660nm')  # - K which is ommitted since it's zero
n1 = np.cross(v_E, v_485) # first plane
n2 = np.cross(v_E, v_660) # second plane

H_tritans_brettel = [
    np.array([
        [1, 0, 0],
        [0, 1, 0],
        [-n1[0]/n1[2], -n1[1]/n1[2], 0]]),
    np.array([
        [1, 0, 0],
        [0, 1, 0],
        [-n2[0]/n2[2], -n2[1]/n2[2], 0]])
]

print()
printMatrix('Projection matrix for tritan (485nm)', H_tritans_brettel[0])

print()
printMatrix('Projection matrix for tritan (660nm)', H_tritans_brettel[1])

Now we can create the simulation function:

In [ ]:
def projection_matrix(plane_normal, anomaly: simulate.Deficiency):
    n = plane_normal
    if anomaly == simulate.Deficiency.PROTAN:
        return np.array([
            [0., -n[1]/n[0], -n[2]/n[0]],
            [0, 1, 0],
            [0, 0, 1] 
        ])
    if anomaly == simulate.Deficiency.DEUTAN:
        return np.array([
            [1, 0, 0],
            [-n[0]/n[1], 0, -n[2]/n[1]],
            [0, 0, 1] 
        ])
    if anomaly == simulate.Deficiency.TRITAN:
        return np.array([
            [1, 0, 0],
            [0, 1, 0],
            [-n[0]/n[2], -n[1]/n[2], 0]
        ])
    return None

def simulate_dichromacy_brettel(im, lms_E, lms_on_wing1, lms_on_wing2, anomaly: simulate.Deficiency):
    n1 = np.cross(lms_E, lms_on_wing1) # first plane
    n2 = np.cross(lms_E, lms_on_wing2) # second plane
    n_sep_plane = np.cross(lms_E, simulate.lms_confusion_axis(anomaly)) # separation plane going through the diagonal
    # Swap the input so that wing1 is on the positive side of the separation plane
    if np.dot(n_sep_plane, lms_on_wing1) < 0:
        n1, n2 = n2, n1
        lms_on_wing1, lms_on_wing2 = lms_on_wing2, lms_on_wing1
    
    H1 = projection_matrix(n1, anomaly)
    H2 = projection_matrix(n2, anomaly)
    
    im_linear_rgb = convert.linearRGB_from_sRGB(convert.as_float32(im))
    im_lms = convert.apply_color_matrix(im_linear_rgb, lms_model.LMS_from_linearRGB)
    im_H1 = convert.apply_color_matrix(im_lms, H1)
    im_H2 = convert.apply_color_matrix(im_lms, H2)
    H2_indices = np.dot(im_lms, n_sep_plane) < 0

    # Start with H1, then overwrite the pixels that are closer to plane 2 with im_H2
    im_H = im_H1
    im_H[H2_indices] = im_H2[H2_indices]
    im_linear_rgb = convert.apply_color_matrix(im_H, lms_model.linearRGB_from_LMS)
    return convert.as_uint8(convert.sRGB_from_linearRGB(im_linear_rgb))

im_tritan_brettel = simulate_dichromacy_brettel(im, lms_of_key(KBMRGCWY, 'E'), lms_485, lms_660, simulate.Deficiency.TRITAN)
cv2.putText(im_tritan_brettel, 'Tritan (Brettel)', (125,190), cv2.FONT_HERSHEY_COMPLEX, 0.5, (64,64,64), 1, cv2.LINE_AA)
px.imshow(np.hstack([im_original, im_tritan, im_tritan_brettel])).update_xaxes(showticklabels=False).update_yaxes(showticklabels=False)

In [ ]:
v_E = lms_of_key(KBMRGCWY, 'E') # - K which is ommitted since it's zero
v_475 = lms_of_key(KBMRGCWY, '475nm') # - K which is ommitted since it's zero
v_575 = lms_of_key(KBMRGCWY, '575nm')  # - K which is ommitted since it's zero
n1 = np.cross(v_E, v_475) # first plane
n2 = np.cross(v_E, v_575) # second plane

H_protans_brettel = [
    np.array([
        [0., -n1[1]/n1[0], -n1[2]/n1[0]],
        [0, 1, 0],
        [0, 0, 1]]),
    np.array([
        [0., -n2[1]/n2[0], -n2[2]/n2[0]],
        [0, 1, 0],
        [0, 0, 1]])]

H_deutans_brettel = [
    np.array([
        [1, 0, 0],
        [-n1[0]/n1[1], 0, -n1[2]/n1[1]],
        [0, 0, 1]]),
    np.array([
        [1, 0, 0],
        [-n2[0]/n2[1], 0, -n2[2]/n2[1]],
        [0, 0, 1]])
]

print()
printMatrix('Projection matrix for protan (475nm)', H_protans_brettel[0])
printMatrix('Projection matrix for protan (575nm)', H_protans_brettel[1])

print()
printMatrix('Projection matrix for deutan (475nm)', H_deutans_brettel[0])
printMatrix('Projection matrix for deutan (575nm)', H_deutans_brettel[1])

In [ ]:
im_protan_brettel = simulate_dichromacy_brettel(im, lms_E, lms_475, lms_575, simulate.Deficiency.PROTAN)
cv2.putText(im_protan_brettel, 'Protan (Brettel)', (125,190), cv2.FONT_HERSHEY_COMPLEX, 0.5, (64,64,64), 1, cv2.LINE_AA)
px.imshow(np.hstack([im_original, im_protan, im_protan_brettel])).update_xaxes(showticklabels=False).update_yaxes(showticklabels=False)

In [ ]:
im_deutan_brettel = simulate_dichromacy_brettel(im, lms_E, lms_475, lms_575, simulate.Deficiency.DEUTAN)
cv2.putText(im_deutan_brettel, 'Deutan (Brettel)', (125,190), cv2.FONT_HERSHEY_COMPLEX, 0.5, (64,64,64), 1, cv2.LINE_AA)
px.imshow(np.hstack([im_original, im_deutan, im_deutan_brettel])).update_xaxes(showticklabels=False).update_yaxes(showticklabels=False)

For protanopia deuteranopia the results are much more similar, so the simpler Viénot approach is a good choice. It also better preserves with extreme values like pure RGB white (255,255,255) since the plane diagonal goes through it.

## How accurate are these simulations?

Both the {% cite vienot_digital_1999 %} and {% cite brettel_computerized_1997 %} methods have fairly a solid background, but they remain approximate mathematical models of the complex human perception of colors. So how good are they at simulating how a person with CVD perceives colors? Several studies have attempted to validate these experimentally. A first step is to ask a person with CVD to validate whether the original image and the simulated image look similar, which is what {% cite brettel_computerized_1997 %} did. More elaborate studies compared how people with CVD and people with normal vision performed on color tests or color-related tasks. On the original images people with normal vision are expected to perform better, but when given the simulated images they should perform similarly as the people with the corresponding CVD, if the simulate is accurate. {% cite simon_liedtke_using_2016 %} show that the Brettel model is pretty good at least for deuteranopes, and {% cite machado_physiologically_based_2009 %} showed pretty good results for protanopes and deuteranopes for their method, which is similar to the Brettel one for full dichromacy.

There are still some serious limits though, so all these models have to be taken with a grain of salt:

- So far we've mostly discussed dichromacy, where one kind of cone is entirely missing. But most people actually have anomalous trichromacy, where the deficient cones are still either present but with less density, or shifted towards another cone, limiting its discrimitive power. For those the dichromat models will look too extreme as they can perceive more colors and see a difference between the original and simulated images. We'll discuss that next.

- All these models are based on _average_ observers. But there are great individual variations. For example {% cite brettel_computerized_1997 %} noted that some experiments on 4 deuteranopes showed that two had a spectral peak of 558 nm for two of them, and 563nm for the other two. So ideally the parameters of the models would need to get adjusted accordingly for each person.

- These models assumed that only the lowest level of color perception is affected, and ignore the plasticity of the brain that can potentially adapt and change the color perception at higher levels. These simulations do not model anything like that.

- The validation experiments are generally made on very small population, rarely more than 10 people. And tritanopes are basically never evaluated since they are very rare and hard to find.

- Most people will look at these simulations on an uncalibrated computer screen in a room with some kind of background illumination. Without a proper calibration of the monitor the stimuli that corresponds to each sRGB value will be innaccurate, and the background illumination may be far from D65 (normal outdoor). Also the brightness, contrast and color balance settings of the monitor can significantly change the color appearance.

- {% cite meyer_color_defective_1988 %} noted that the size of the field of view can affect the severity of an individual's deficiency. They refer to {% cite pokorny_new_1982 %} that observed that the severity can decrease as the field of view increases. This explains why colors are harder to identify for a person with CVD on small objects like LEDs, but easier on large bright objects. Ideally the simulations should take that into account and preserve more of the original color on large objects.

# What about anomalous trichromacy?

As mentioned above, most people with CVD still have _some_ perception with the deficient kind of cones. The {% cite vienot_digital_1999 %} and {% cite brettel_computerized_1997 %} models can be adjusted to be more progressive by not projecting all the way towards the plane.

{%cite flatla_so_2012} (refined for a mobile app in {%cite macalpine_real_time_2016}) propose to make a fixed step towards the plane, with the step size determined from a custom calibration. The step is actually made in the L*u*v color space since a fixed step there is supposed to be perceptually uniform in that space.

Another approach was proposed was {% cite machado_physiologically_based_2009 %}, where the degree of severity comes from an amount of shift in the peak wavelength of the cone response. Dichromacy can be represented when e.g. the M-cone is shifted to align with the S-cone, effectively suppressing it. By shifting it less, various degrees of severity can be simulated. At the end of the day {%cite macalpine_real_time_2016} mentions that their results are quite similar, the main difference being that with {% cite machado_physiologically_based_2009 %} the step size is proportional to the distance between the point and the plane, while the steps are fixed in {%cite macalpine_real_time_2016}. It's unclear which one is more accurate from a perception point of view.

It's also trivially possible to extend the Viénot or Brettel methods to have a severity factor between 0 and 1 that does a linear interpolation between the dichromat simulation and the original image. In practice this gives pretty good results, making {% cite brettel_computerized_1997 %} and {% cite machado_physiologically_based_2009 %} quite close for protanomaly and deuteranomaly. Here is an example for protanomaly:

<!-- Image tags have to be unindented for nbdev to catch them :-( -->
<table>
    <tr><th>Original</th><th>Machado 2009 (1.0 severity)</th><th>Brettel 1997 (1.0 severity)</th></tr>
    <tr><td>
<img src="simulation_images/rgbspan.png"></td><td>
<img src="simulation_images/rgbspan_protan_machado2009.png"></td><td>
<img src="simulation_images/rgbspan_protan_brettel1997.png"></td></tr>
     <tr><th>Original</th><th>Machado 2009 (0.5 severity)</th><th>Brettel 1997 (0.5 severity)</th></tr>
    <tr><td>
<img src="simulation_images/rgbspan.png"></td><td>
<img src="simulation_images/rgbspan_protan_machado2009_0.5.png"></td><td>
<img src="simulation_images/rgbspan_protan_brettel1997_0.5.png"></td></tr>
</table>

The main problem then becomes to estimate the severity factor for each individual. {%cite flatla_so_2012} uses a calibration method where a number of colored patterns are shown to the user with an increasing distance from the neutral gray until she can distinguish them. And that distance is then used to determine the severity factor. But we're still lacking convenient opensource tools to do that, so in practice people just have to try various parameters until they can't see the diffence between the original and the simulated images.

# Conclusion

So the good news is that there are quite a few solutions to simulate CVDs. The bad news is there is still a lot of outdated code out, and even for well-intentioned developers it's quite hard to evaluate whether the output of their simulation is correct or not, so I hope that this article can help.

A second issue is that the methods that are easy to implement are generic and were developed from experimental data collected on average observers (and a long time ago!). We're still lacking a great practical way to model individual color vision deficiency profiles and feed that into a simulator. The work of {% cite flatla_so_2012 %} and {% cite macalpine_real_time_2016 %} goes in that direction, but as far as I know there is no opensource code for it. If you want to evaluate what kind of blindness and what severity you have according to the LMS models discussed in this article I've generated a number of Ishihara-like plates in the Appendix FIXME.

There are also other research work that have remained more confidential so far like {% cite capilla_corresponding_pair_2004 %}. While interesting, the main problem here again is that no large-scale validation was performed, so it's hard to know which one is better. That group also did share [some Matlab code](http://rua.ua.es/dspace/handle/10045/23471), but it hasn't been maintained and can't be run without a Matlab runtime from 2011.

The next step after simulation is to try to transform images to help people with CVD to process them. The most well-known approach is the daltonize algorithm from {% cite fidaner_analysis_2005 %}, but it's main advantage is the simplicity and code availability as it is quite rough (it was actually just a small school project!). There are a lot of more recent work in that area, so it'll be the subject of a followup article.

# Appendix

## Code to generate the confusion lines of the introduction

In [ ]:
# Debug figure to visualize the confusion lines
fig = px.scatter_3d(KBMRGCWY.loc['K':'Y'], x='L', y='M', z='S', 
                    color='colorhex', color_discrete_map='identity', text='short_name',
                    width=800, height=800)

# Add the parallelepiped lines
df = KBMRGCWY.loc[['K','B','M','R', 'K', 'G', 'Y', 'W', 'C', 'G', 'Y', 'R', 'M', 'W', 'C', 'B']]
fig.add_scatter3d(x=df['L'], y=df['M'], z=df['S'], mode='lines', showlegend=False)

# Add the black-white diagonal
df = KBMRGCWY.loc[['K','W']]
fig.add_scatter3d(x=df['L'], y=df['M'], z=df['S'], mode='lines', showlegend=False, line=dict(width=8, dash='dash', color='gray'))

# add_black_white_yellow_blue_plane(fig, KBMRGCWY)
   
def generate_confusion_line(lms_color, anomaly, num_steps, fig):
    # This returns a line segment along the anomaly projection axis (L, M or S)
    # that passes through the input LMS color and stops at the boundaries of the
    # gamut (the parallelepiped). Then we can walk along that segment with small
    # steps to generate confusion colors.
    segment = geometry.lms_confusion_segment(lms_color, lms_model, anomaly)
    colors = []
    # Ensure num_steps at the end
    step = (1.0 / (num_steps-1)) - 1e-5
    for s in np.arange(0.0, 1.0, step):
        p = segment[0]*(1.0-s) + segment[1]*s
        colors.append (p)
        if fig:
            fig.add_scatter3d(x=[p[0]], y=[p[1]], z=[p[2]], showlegend=False,
                              marker=dict(color=lms_model.sRGB_hexstring_from_LMS(p), 
                                          line=dict(color='Black', width=1)))
    return colors
    
anomaly = simulate.Deficiency.PROTAN
num_steps = 7

# Generate segments that passes through a set of LMS points. To get a nice
# distribution of the segment we sample them from the diagonal Black-Yellow-Blue plane
U = lms_of_key(KBMRGCWY, 'Y') - lms_of_key(KBMRGCWY, 'K')
V = lms_of_key(KBMRGCWY, 'B') - lms_of_key(KBMRGCWY, 'K')
confusion_lines = []
# for uv_plane in itertools.product([0.1,0.25,0.5,0.75,0.9], [0.02,0.1,0.25,0.5,0.75,0.9]):
for uv_plane in [(0.2,0.2), (0.5,0.5), (0.25, 0.75), (0.75, 0.25), (0.8, 0.8)]:
    lms_color = uv_plane @ np.array([U,V])
    colors = generate_confusion_line(lms_color, anomaly, num_steps, fig)
    confusion_lines.append(colors)

print (f"# Confusion lines for {anomaly}")
print ("confusion_lines = [")
for line in confusion_lines:
    print (" "*4 + "pd.DataFrame.from_records([")
    lines_as_text = []
    for c in line:
        srgb_hex = lms_model.sRGB_hexstring_from_LMS(c)
        L, M, S = c[0], c[1], c[2]
        linear_rgb = convert.apply_color_matrix(c, lms_model.linearRGB_from_LMS)
        linear_rgb = np.clip(linear_rgb,0.,1.0)
        R, G, B = linear_rgb[0], linear_rgb[1], linear_rgb[2]
        lines_as_text.append(" "*8 + f"({L:.4f}, {M:.4f}, {S:.4f}, {R:.4f}, {G:.4f}, {B:.4f}, '{srgb_hex}')")
    print (",\n".join(lines_as_text) + "],")    
    print (" "*8 + "columns = ['L', 'M', 'S', 'R', 'G', 'B', 'sRGB_hex']),")
print("]")

fig.show()

confusion_matrices = []
for confusion_line in confusion_lines:
    confusion_matrix = np.expand_dims(np.array(confusion_line), axis=0)
    confusion_matrix = convert.apply_color_matrix(confusion_matrix, lms_model.linearRGB_from_LMS)
    confusion_matrix = convert.sRGB_from_linearRGB(confusion_matrix)
    confusion_matrices.append(np.clip(confusion_matrix*255.0, 0., 255.).astype(np.uint8))
fig = px.imshow(np.vstack(confusion_matrices), title=f"Confusion lines for {anomaly}. The colors on each row should appear similar to a dichromat.").update_layout(hide_image_axes)
fig.show()

## Generated Ishihara-like plates to evaluate the kind and severity of CVD

It is possible to generate Ishihara-like plates for a given deficiency and severity factor. The main idea is to pick a confusion segment in the LMS space, and generate an Ishihara-like image by using one end of the segment as the background color and the other end as the foreground color (a single-digit number). On top of that lower degrees of severity can be tested by making the segment shorter, effectively reducing the distance between the colors along the confusion line. This makes it harder and harder to differentiate for someone with anomalous trichromacy.

By looking at these generated plates it becomes possible to self-evaluate the kind of deficiency by checking in which plate the numbers are the harder to read. For example I can read all the numbers in the tritan plate, most numbers in the deutan plate (but not all), and barely any in the protan plate. This confirms that I am a protan.

Then the severity can be evaluated by looking at plates generated with decreasing severity and checking at which value it becomes impossible to see any number. For me that's around 0.8 for the protan plate.

In [ ]:
def showAnimatedImages(title, deficiency):
    images = []
    for severity in np.arange(1.0, 0.19, -0.1):
        im = generate.ishihara_plate(deficiency, severity, f"Severity {severity:.1f}")
        images.append (im)
    images = np.stack(images, axis=0)
    fig = px.imshow(images, height=704*1.2, animation_frame=0, title=title).update_layout(hide_image_axes)
    fig.show()

showAnimatedImages("Protanopia / protanomaly", simulate.Deficiency.PROTAN)
showAnimatedImages("Deuteranopia / deuteranomaly", simulate.Deficiency.DEUTAN)
showAnimatedImages("Tritanopia / tratanomaly", simulate.Deficiency.TRITAN)

{% bibliography --cited %}

In [ ]:
#hide
# The cell above will show a nice bibliography in the HTML output. This cell just shows the raw bibtex source.
# !cat ../references.bib